In [31]:
import pandas as pd
import numpy as np
import gc
import os
import shutil
from tqdm import tqdm

In [4]:
basedir = './국토교통부_실거래가_공개시스템/'
filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv')|(f.endswith('.xlsx')))]
filenames

['아파트(매매)__실거래가_20201211150806.csv',
 '아파트(매매)__실거래가_20201211152927.csv',
 '아파트(매매)__실거래가_20201214122854.csv',
 '아파트(매매)__실거래가_20201214124757.csv',
 '아파트(매매)__실거래가_20201214133432.csv',
 '아파트(매매)__실거래가_20201214141339.csv',
 '아파트(매매)__실거래가_20201214145855.csv',
 '아파트(매매)_실거래가_20201211124340.xlsx',
 '아파트(매매)_실거래가_20201211125314.xlsx',
 '아파트(매매)_실거래가_20201211133311.xlsx',
 '아파트(매매)_실거래가_20201211134257.xlsx',
 '아파트(매매)_실거래가_20201211135205.xlsx',
 '아파트(매매)_실거래가_20201211142038.xlsx',
 '아파트(매매)_실거래가_20201211142802.xlsx',
 '아파트(매매)_실거래가_20201211155645.xlsx',
 '아파트(전월세)_실거래가_20201211143741.xlsx',
 '아파트(전월세)_실거래가_20201211150050.csv',
 '아파트(전월세)_실거래가_20201211152214.csv',
 '아파트(전월세)_실거래가_20201211153717.csv',
 '아파트(전월세)_실거래가_20201214123343.csv',
 '아파트(전월세)_실거래가_20201214124257.csv',
 '아파트(전월세)_실거래가_20201214130430.csv',
 '아파트(전월세)_실거래가_20201214140753.csv',
 '아파트(전월세)_실거래가_20201214144600.csv',
 '아파트(전월세)_실거래가_20201214181419.csv',
 '연립다세대(매매)__실거래가_20201211150924.csv',
 '연립다세대(매매)__실거래가_20201211153058.cs

In [23]:
filenames[0]

'아파트(매매)__실거래가_20201211150806.csv'

In [5]:
f = open(basedir + filenames[0])

In [6]:
f.readline()

'"□ 본 서비스에서 제공하는 정보는 법적인 효력이 없으므로 참고용으로만 활용하시기 바랍니다."\n'

In [7]:
f.readline()

'"□ 신고정보가 실시간 변경, 해제되어 제공시점에 따라 공개건수 및 내용이 상이할 수 있는 점 참고하시기 바랍니다."\n'

In [8]:
f.readline()

'"□ 본 자료는 계약일 기준입니다. (※ 7월 계약, 8월 신고건 → 7월 거래건으로  제공)"\n'

In [9]:
f.readline()

'"□ 통계자료 활용시에는 수치가 왜곡될 수 있으니 참고자료로만 활용하시기  바라며,  외부 공개시에는 반드시 신고일 기준으로 집계되는 공식통계를 이용하여 주시기 바랍니다."\n'

In [10]:
f.readline()

'""\n'

In [11]:
f.readline()

'"* 국토교통부 실거래가 공개시스템의 궁금하신 점이나 문의사항은 콜센터 1588-0149로 연락 주시기 바랍니다."\n'

In [12]:
f.readline()

'"□ 검색조건"\n'

In [13]:
f.readline()

'"계약일자 : 20130101 ~ 20131231"\n'

In [14]:
f.readline()

'"실거래구분 : 아파트(매매)"\n'

In [15]:
f.readline()

'"주소구분 : 지번주소"\n'

In [16]:
f.readline()

'"시도 : 서울특별시"\n'

In [17]:
f.readline()

'"시군구 : 전체"\n'

In [18]:
f.readline()

'"읍면동 : 전체"\n'

In [19]:
f.readline()

'"면적 : 전체"\n'

In [20]:
f.readline()

'"금액선택 : 전체"\n'

In [21]:
f.readline()

'"시군구","번지","본번","부번","단지명","전용면적(㎡)","계약년월","계약일","거래금액(만원)","층","건축년도","도로명"\n'

In [22]:
f.readline()

'"서울특별시 강남구 개포동","655-2","0655","0002","개포2차현대아파트(220)","77.75","201309","8","57,000","2","1988","언주로 103"\n'

In [24]:
f.close()

In [49]:
for f in tqdm(filenames):
    tradetype = ''
    date = ''
    
    try:
        file_open = open(basedir + f, 'r')        
        line = file_open.readline()
    except:
        try:
            file_open.close()
            file_open = open(basedir + f, 'r', encoding='euc-kr')
            line = file_open.readline()
        except:
            try:
                file_open.close()
                file_open = open(basedir + f, 'r', encoding='ansi')
                line = file_open.readline()
            except:
                file_open.close()
                file_open = open(basedir + f, 'r', encoding='utf-8')
                line = file_open.readline()
    
    while tradetype == '':
        line = file_open.readline()        
        
        if '계약일자' in line:
            date = line.replace('계약일자', '').replace(':', '').replace('\"', '').replace('\\', '').replace('n', '')\
            .replace(' ', '').replace('~', '_').replace('\n', '')
        
        if '실거래구분' in line:
            tradetype = line.replace('실거래구분', '').replace('\"', '').replace(':', '').replace('\\', '')\
            .replace('n', '').replace(' ', '').replace('\n', '')
            
            file_open.close()
    
    if tradetype == 'error':
        continue
    
    original = basedir + f
    if f.endswith('.csv'):
        target = basedir + tradetype + '_' + date + '.csv'
    elif f.endswith('.xlsx'):
        target = basedir + tradetype + '_' + date + '.xlsx'
    
    shutil.copyfile(original, target)

  9%|███████▋                                                                          | 7/75 [00:00<00:00, 101.43it/s]


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 12: invalid start byte

In [50]:
file_open.close()